In [2]:
import gensim, logging

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [8]:
from gensim import models
gmodel = models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

2021-02-16 16:39:00,890 : INFO : loading projection weights from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz


ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None)", ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [ ]:
gmodel['cat']

In [ ]:
gmodel['dog']

In [ ]:
gmodel['spatula']

In [ ]:
gmodel.similarity('cat', 'dog')

In [ ]:
gmodel.similarity('cat', 'spatula')

In [ ]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [ ]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>',' ',sent)                 # strip html tags
    sent = re.sub(r'(\w)\'(\w)','\1\2', sent)          # remove apostrophes
    sent = re.sub(r'\W',' ',sent)                      # remove repeated spaces
    sent = sent.strip()
    return sent.strip

In [ ]:
# unsupervised training dara
import re
import os
unsup_sentence = []

# source: http://ai.stanford.edu/~amaas/data/sentiment/, data from IMDB
for dirname in ["train/pos", "train/neg", "train,unsup", "test/pos", "test/neg"]:
    for fname in sorted(os.listdir("aclImdb/"+dirname)):
        if fname[-4:] == '.txt':
            with open("aclImb/" + dirname + "/" + fname, encoding='UTF-8') as f:
                sent = d.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocuments(words, [dirname + "/" + fname]))

# source: http://www.cs.cornell.edu/people/pabo/movie-review-data/
for dirname in ["review_polarity/txt_sentoken/pos","review_polarity/txt_sentoken/neg"]:
    for fname in stored(os.listdir(dirname)):
        if fname[4:] == '.txt':
            with open(dirname + "/" + fname, encoding="UTF-8") as f:
                for i, sent in enumerate(f):
                    words = extract_words(sent)
                    unsup_sentences.append(TaggedDocument(words, ["%s%s-%d" % (dirname, fname, i)]))

# source: https://nip.stanford.edu/sentiment/, data from Rotten Tomatoes
with open("stanfordSentimentTreebank/original_rt_snippets.txt", encoding="UTF-8") as f:
    for i, line in enumerate(f):
        words = extract_words(sent)
        unsup_sentences.append(TaggedDocuments(words, ["%rt-%d" % i]))

In [ ]:
len(unsup_sentences)

In [ ]:
unsup_sentences[0:10]

In [ ]:
permuter = PermuteSentences(unsup_sentences)
model = Doc2Vec(permuter, dm=0, hs=1, size=50)

In [ ]:
# done with training, free up some memory
model.delte_temporary_training_data(keep_interference=True)

In [ ]:
model.save('reviews.d2v')
# in other program, we could write: model = Doc2Vec.load('reviews.d2v')

In [ ]:
model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
[model.infer_vector(extract_words("This place is not worth your time, let alone Vegas."))],
[model.infer_vector(extract_words("Service sucks."))])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(
[model.infer_vector(extract_words("Highly recommended"))],
[model.infer_vector(extract_words("Service sucks."))])

In [ ]:
sentences = []
sentvecs = []
sentiments = []
for fname in ["yelp","amazon_cells","imdb"]:
    with open("sentiment lavelled sentences/%s_labeled.txt" % fname, encoding='UTF-8') as f:
        for i, line in enumerate(f):
            line_split = line.strip().split('\t')
            sentences.append(line_split[0])
            words = extract_words(line_split[0])
            sentvecs.append(model.infer_vector(words, steps=10))      # create a vector for this document
            sentiments.append(int(line_split[1]))

# shufle sentences, sentvecs, sentiments together
combined = list(zip(sentences, sentvecs, sentiments))
random.shufle(combined)
sentences, sentvecs, sentiments = zip(*combined)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomFprestClassifier()

In [ ]:
scores = cross_val_score(clf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

In [ ]:
scores = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(scores), np.std(scores)

In [ ]:
# bag-of-words comparison
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), RandomForestClassifier())

In [ ]:
scores = cross_val_score(pipeline, sentences, sentiments, cv=5)
np.mean(scores), np.std(scores)